# KPop song clustering

### Objective: Used unsupervised learning to cluster korean pop songs that I like (in playlist). Compare with Anime playlist. Compare with "iconic" KPop playlist

### Methodology: 1) Create dataset with track name, artist, audio features, playlist name, like y/n

In [255]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import time
import pandas as pd
import numpy as np

In [345]:
# This cell is hidden
#Authorize credentials
cid = "c433ddf327d64bbcbae13b46dc5a56c9" 
secret = "ed9ffb3511ff4bb7910d520564ddbd79" 
user_id = '22pjtm23jl5h4qq4srfwxwxqa' #my playlists
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [189]:
def getplayListDict(user_id):
    playlists = sp.user_playlists(user_id)
    names = [x['name'] for x in playlists['items']]
    ids = [x['id'] for x in playlists['items']]
    return dict(zip(names, ids))

In [202]:
def getTrackInfoFromPlaylist(user_id, playlist_id):
    names, ids, artists = [], [], []
    tracks = sp.user_playlist_tracks(user_id, playlist_id)
    for track in tracks['items']:
        names.append(track['track']['name'])
        ids.append(track['track']['id'])
        artists.append(track['track']['artists'][0]['name'])
    df = pd.DataFrame({'Track Name': names, 'Track ID': ids, 'Artist': artists})
    return df

In [203]:
my_playlist = getplayListDict(user_id)

In [212]:
pl_list = []
for playlist in my_playlist.keys():
    playlist_df = getTrackInfoFromPlaylist(user_id, my_playlist[playlist])
    playlist_df['Playlist'] = playlist
    pl_list.append(playlist_df)
tracks_df = pd.concat(pl_list)

In [245]:
#remove none
tracks_df.dropna(inplace=True)
tracks_df

,Track Name,Track ID,Artist,Playlist
0,HELICOPTER,2hJS7BdEaH9Jo8MeJujsHT,CLC,KPOP
1,HANN (Alone),7gr57cYekMWriyJYbT7oZ4,(G)I-DLE,KPOP
2,Oh my god,2DmRXiyn03tOqKgEJXlaiJ,(G)I-DLE,KPOP
3,HWAA,5FiXhM80sP4yg6tEnHkZZn,(G)I-DLE,KPOP
4,LION,40OyiVO9NtBg9R2Gpwxs3u,(G)I-DLE,KPOP
...,...,...,...,...
95,One Small Step,0wvi4qO9vQ0HkkKrMAdoTA,リリアン・ワインバーグ(Song Performed by Laura Pitt-Pulford),lit anime music
96,楽園,1MFLgJxf4vcLxXubNlJE54,Fujifabric,lit anime music
97,Give It Up?,6dJqNfKmEz64garAEuZBtp,Kuroneko,lit anime music
98,IKOUZE PARADISE,6PWTHFbYE9wSaQojr7umQ5,STUNK(CV:JUNJI MAJIMA),lit anime music


In [316]:
def getAudioFeatures(track_id_list, limit=50, sleeptime=30):
    features = []
    n_calls = 0
    for track_id in track_id_list:
        try:
            feature = sp.audio_features([track_id])
            features.append(feature)
            n_calls+=1
        except:
            features.append("Fail")
            n_calls+=1
        if n_calls>50:
            time.sleep(sleeptime)
            n_calls = 0
    return features

In [317]:
audio_features = getAudioFeatures(tracks_df['Track ID'].tolist())
print(set([x[0]['id'] for x in audio_features] == tracks_df['Track ID']))

{True}


In [319]:
audio_feature_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

In [333]:
def cleanFeaturesDF(audio_features, audio_feature_columns):
    df_list = []
    for song in audio_features:
        song_df = pd.DataFrame.from_dict(song)[['id'] + audio_feature_columns]
        df_list.append(song_df)
    features_concat = pd.concat(df_list)
    return features_concat

In [337]:
features_cleaned.head()

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2hJS7BdEaH9Jo8MeJujsHT,0.666,0.908,5,-2.819,0,0.0560,0.0323,0.000000,0.3830,0.655,155.048,225720,4
0,7gr57cYekMWriyJYbT7oZ4,0.709,0.718,6,-3.315,0,0.0312,0.0590,0.000000,0.1670,0.240,104.971,205820,4
0,2DmRXiyn03tOqKgEJXlaiJ,0.794,0.762,4,-3.525,0,0.0392,0.0358,0.000001,0.1910,0.482,109.970,195867,4
0,5FiXhM80sP4yg6tEnHkZZn,0.384,0.868,6,-3.226,0,0.0711,0.0458,0.000006,0.0797,0.233,95.027,197320,4
0,40OyiVO9NtBg9R2Gpwxs3u,0.741,0.715,4,-5.206,0,0.0340,0.0318,0.000000,0.1210,0.187,109.989,210973,4


In [341]:
df_merged_features = pd.merge(tracks_df, features_cleaned, how='left', left_on='Track ID', right_on = 'id')
df_merged_features = df_merged_features.drop(columns=['id'])

In [344]:
df_merged_features

,Track Name,Track ID,Artist,Playlist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,HELICOPTER,2hJS7BdEaH9Jo8MeJujsHT,CLC,KPOP,0.666,0.908,5,-2.819,0,0.0560,0.03230,0.000000,0.3830,0.655,155.048,225720,4
1,HANN (Alone),7gr57cYekMWriyJYbT7oZ4,(G)I-DLE,KPOP,0.709,0.718,6,-3.315,0,0.0312,0.05900,0.000000,0.1670,0.240,104.971,205820,4
2,Oh my god,2DmRXiyn03tOqKgEJXlaiJ,(G)I-DLE,KPOP,0.794,0.762,4,-3.525,0,0.0392,0.03580,0.000001,0.1910,0.482,109.970,195867,4
3,HWAA,5FiXhM80sP4yg6tEnHkZZn,(G)I-DLE,KPOP,0.384,0.868,6,-3.226,0,0.0711,0.04580,0.000006,0.0797,0.233,95.027,197320,4
4,LION,40OyiVO9NtBg9R2Gpwxs3u,(G)I-DLE,KPOP,0.741,0.715,4,-5.206,0,0.0340,0.03180,0.000000,0.1210,0.187,109.989,210973,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,One Small Step,0wvi4qO9vQ0HkkKrMAdoTA,リリアン・ワインバーグ(Song Performed by Laura Pitt-Pulford),lit anime music,0.323,0.367,7,-8.151,1,0.0385,0.77000,0.000000,0.0942,0.332,81.909,251098,4
153,楽園,1MFLgJxf4vcLxXubNlJE54,Fujifabric,lit anime music,0.566,0.826,2,-5.086,1,0.0732,0.00457,0.000002,0.0949,0.443,97.937,225560,4
154,Give It Up?,6dJqNfKmEz64garAEuZBtp,Kuroneko,lit anime music,0.725,0.816,6,-6.024,0,0.0563,0.02070,0.000003,0.0543,0.709,89.974,168480,4
155,IKOUZE PARADISE,6PWTHFbYE9wSaQojr7umQ5,STUNK(CV:JUNJI MAJIMA),lit anime music,0.599,0.963,6,-1.967,1,0.1150,0.06620,0.000000,0.3320,0.554,141.058,255175,4
